In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from keplergl import KeplerGl
from datetime import datetime
import time
import os
import h3
from shapely.geometry import shape, Point
from turfpy.transformation import convex

%matplotlib inline

In [2]:
# Inrix weekday-codes
dow_dict = {'Montag':0, 'Dienstag':1, 'Mittwoch':2, 'Donnerstag':3, 'Freitag':4, 'Samstag':5, 'Sonntag':6}

In [3]:
files_0 = os.listdir('./data/FCD/')
dates = list(set([x.split('_')[0] for x in files_0]))
dates.sort()

# define grid size, for more information see: https://h3geo.org/docs/core-library/restable
APERTURE_SIZE = 11 # Average edge length: 0.028663897 km
hex_col = 'hex'+str(APERTURE_SIZE)

plz = pd.read_json('data/Forchheim_plz.json')

In [4]:
# filter for specific weekdays
def check_dates(dates,date_filter):
    date_query = []
    date_filter = [dow_dict[x] for x in date_filter]

    for date in dates:
        dow = datetime(int(date[:4]),int(date[4:6]),int(date[6:])).weekday()
        if dow in date_filter:
            date_query.append(date)
        else:
            pass
    return date_query

In [5]:
# filter for specific times
def time_filter(df,start,end):
    df['hour'] = df['timestamp'].apply(lambda x: x.hour)
    df = df[(df['hour'] >= start) & (df['hour']<end)]
    df = df.drop(['hour'],axis=1)
    df.reset_index(drop=True,inplace=True)
    return df

In [6]:
# load and concat all files per day
def day_load(date):
    date_list = [s for s in files_0 if date in s]
    df = pd.read_csv('data/FCD/' + date_list[1], compression='gzip', header=None, sep=',')
    headers = ['session_id', 'speed', 'timestamp', 'latitude', 'longitude', 'direction']
    df.columns = headers
    for i in range(1,len(date_list)):
        df_i = pd.read_csv('data/FCD/' + date_list[i], compression='gzip', header=None, sep=',')
        headers = ['session_id', 'speed', 'timestamp', 'latitude', 'longitude', 'direction']
        df_i.columns = headers
        df = pd.concat([df,df_i], axis=0, ignore_index=True)
    return df

In [7]:
def polygon_check(lon, lat):
    polygon = shape(plz['geometry'][0]['geometry'])
    point = Point(lon, lat)
    return polygon.contains(point)

In [8]:
# filter trips by location
def plz_filter(df,plz):
    lng_min, lng_max, lat_min, lat_max = 100, 0, 100, 0
    plz_shape = plz['geometry'][0]['geometry']['coordinates'][0]
    for i in plz_shape:
        if i[0] < lng_min:
            lng_min = i[0]
        elif i[0] > lng_max:
            lng_max = i[0]
        else:
            pass
        if i[1] < lat_min:
            lat_min = i[1]
        elif i[1] > lat_max:
            lat_max = i[1]
        else:
            pass

    df_plz = df[df['longitude'] > lng_min]
    df_plz = df_plz[df_plz['longitude'] < lng_max]
    df_plz = df_plz[df_plz['latitude'] > lat_min]
    df_plz = df_plz[df_plz['latitude'] < lat_max]

    df_plz['Forchheim'] = df_plz.apply(lambda x: polygon_check(x['longitude'], x['latitude']), axis=1)
    df_plz = df_plz[df_plz['Forchheim']]
    try:
        df_plz.drop(['Forchheim'], axis=1, inplace=True)
    except:
        pass
    return df_plz

In [9]:
# ... the actual grid mapping
def h3_mapping(df, hex_col, cutoff):

    df[hex_col] = df.apply(lambda x: h3.geo_to_h3(x.latitude,x.longitude,APERTURE_SIZE),1)

    df_speed = df.groupby(hex_col)['speed'].mean().to_frame('speed').reset_index()
    df_count = df.groupby(hex_col).size().to_frame('count').reset_index()

    #df_hex['count'] = df.groupby(hex_col).size().to_frame('count').reset_index()['count']
    df_hex = pd.merge(df_speed,df_count,on=hex_col)
    df_hex['speed'] = df_hex['speed'].apply(lambda x: round(x,1))

    df_hex = df_hex[df_hex['count'] >= cutoff].reset_index(drop=True)

    #df_hex = df.groupby(hex_col)['speed'].mean().to_frame('speed').reset_index()
    return df_hex

In [10]:
# main function
def data_transform(date_query,time_range,cutoff=0):
    hex_gesamt = pd.DataFrame([])
    dataframes = {}
    for i in range(len(time_range)-1):
        dataframes['hex_'+str(i)] = pd.DataFrame([])

    for date in date_query:
        df = day_load(date)
        
        df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
        df['unix'] = df['timestamp'].apply(lambda x: time.mktime(x.timetuple()))

        try:
            df = plz_filter(df,plz)
            df = df.sort_values(['session_id', 'unix'])
            df = df.drop_duplicates(['session_id', 'latitude', 'longitude', 'direction'])
            df.reset_index(drop=True, inplace=True)

            df_hex_i = h3_mapping(df, hex_col, cutoff)
            hex_gesamt = pd.concat([hex_gesamt, df_hex_i], ignore_index=True)

            for i in range(len(time_range)-1):
                start = time_range[i]
                end = time_range[i+1]
                df_i = time_filter(df,start,end)
                df_hex_i = h3_mapping(df_i, hex_col, cutoff)
                dataframes['hex_'+str(i)] = pd.concat([dataframes['hex_'+str(i)], df_hex_i], ignore_index=True)
        except:
            pass
        
        
        #df_hex = df_hex.groupby(hex_col).mean().reset_index()

    speed_df = hex_gesamt.groupby(hex_col)['speed'].mean().reset_index()
    count_df = hex_gesamt.groupby(hex_col)['count'].sum().reset_index()
    hex_gesamt = pd.merge(speed_df,count_df,on=hex_col)

    for i in range(len(time_range)-1):
        speed_df = dataframes['hex_'+str(i)].groupby(hex_col)['speed'].mean().reset_index()
        count_df = dataframes['hex_'+str(i)].groupby(hex_col)['count'].sum().reset_index()
        dataframes['hex_'+str(i)] = pd.merge(speed_df,count_df,on=hex_col)
    #df_hex['speed'] = df_hex['speed'].apply(lambda x: round(x,1))

    #df_hex = df_hex[df_hex['count'] >= cutoff].reset_index(drop=True)
    return hex_gesamt, dataframes

In [11]:
# the cutoff value specifies the minimum data count per cell
cutoff = 20

# time slices for detailed analysis. Each slice includes times between the n and n+1 list entry.
#time_range = [0,6,10,15,19,24]
time_range = [0,12,24]

date_filter = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag']
date_query = check_dates(dates,date_filter)

weekday, weekday_times = data_transform(date_query,time_range,cutoff)

Data export

In [12]:
# optional
weekday.to_csv('data_export/full_day.csv')
df_names = list(weekday_times.keys())
for df_name in df_names:
    df_i = weekday_times[df_name]
    df_i.to_csv('data_export/timeslice_'+ df_name +'.csv')